### 棋盘的实现——来自`Board.py`源代码
修改了一部分，进行适当剪枝提高算法效率

In [140]:
import numpy as np
import random
from collections import deque

BOARD_SIZE = 8
N_ROWS = 1000
COLORS = ['R', 'Y', 'B', 'G']

In [141]:
class MyBoard:
    def __init__(self, board: np.ndarray):
        '''
        Initialize board instance.
        Parameters:
        size(int): Size of each sub-board
        board_num(int): Number of sub-boards in the main board
        colors(np.array): Array of available colors
        '''
        self.size = board.shape[1]
        self.board = board
        self.lazylines = set()

    def change(self, loc1, loc2):
        '''
        Exchange the colors of two adjacent cells in the current sub-board.
        Parameters:
        loc1(tuple): The coordinate (x,y) of one cell to be swapped.
        loc2(tuple): The coordinate (x,y) of another cell to be swapped.
        args: Optional additional parameters that can be ignored.
        Returns:
        None
        '''
        x1, y1 = loc1
        x2, y2 = loc2
        temp1 = self.board[x1, y1].copy()
        temp2 = self.board[x2, y2].copy()
        self.board[x1, y1], self.board[x2, y2] = temp2, temp1
        self.lazylines = {x1, x2}

    def scan_for_connected(self):
        matrix = self.board
        visited = set()
        res = []
        changed = np.zeros((self.size, self.size), bool)

        # Check if it belongs to connected elements
        def check_flag(x, y):
            try:
                if matrix[x, y + 2] == matrix[x, y + 1] == matrix[x, y]:
                    return True
            except IndexError:
                pass
            try:
                if matrix[x + 2, y] == matrix[x + 1, y] == matrix[x, y]:
                    return True
            except IndexError:
                pass
            return False

        # BFS
        for i in self.lazylines.copy():
            for j in range(self.size):
                if (i, j) not in visited:
                    flag = False
                    visited.add((i, j))
                    temp = []
                    queue = deque([(i, j)])

                    while queue:
                        x, y = queue.popleft()
                        if flag or check_flag(x, y):
                            flag = True
                        temp.append((x, y))
                        if x + 1 < self.size and (x + 1, y) not in visited and matrix[x + 1][y] == matrix[x][y]:
                            queue.append((x + 1, y))
                            visited.add((x + 1, y))
                        if x - 1 >= 0 and (x - 1, y) not in visited and matrix[x - 1][y] == matrix[x][y]:
                            queue.append((x - 1, y))
                            visited.add((x - 1, y))
                        if y + 1 < self.size and (x, y + 1) not in visited and matrix[x][y + 1] == matrix[x][y]:
                            queue.append((x, y + 1))
                            visited.add((x, y + 1))
                        if y - 1 >= 0 and (x, y - 1) not in visited and matrix[x][y - 1] == matrix[x][y]:
                            queue.append((x, y - 1))
                            visited.add((x, y - 1))
                    if flag:
                        res.append(temp)
                        self.lazylines |= set(map(lambda x: x[0],temp))
                        for cordinates in temp:
                            changed[cordinates] = True

        return res

    def eliminate(self, func=lambda x: (len(x) - 2) ** 2):
        '''
        Eliminates connected elements from the mainboard and calculates the score.
        Args:
        func (function): A function that takes in a group of connected elements and returns a score.
        Returns:
        tuple: A tuple that contains the total score (int) and the number of columns eliminated (array).
        '''
        # Scan the board for connected elements
        list_of_connected_elements = self.scan_for_connected()

        # Calculate the score for each connected element
        scorelist = list(map(func, list_of_connected_elements))

        # Calculate the total score
        score = sum(scorelist)

        # Mark the connected elements as changed
        changed = np.zeros((self.size, self.size), bool)
        for sublst in list_of_connected_elements:
            for cordinates in sublst:
                changed[cordinates] = True

        # Eliminate the columns with connected elements
        columns_eliminated = np.sum(changed, axis=0)
        for i in range(self.size):
            col = self.board[0:self.size, i]
            indices = np.where(changed[:, i] == False)[0]
            if len(indices) != self.size:
                self.board[:, i] = np.concatenate(
                    (col[indices], self.board[self.size:, i],
                     np.full((columns_eliminated[i],), np.nan)), axis=0)

        # Return the total score and the number of columns eliminated
        return score, columns_eliminated


### 初始化设定
测试，统一设定好种子为40号

### 1. 贪心算法

In [142]:
action_space = [((i, j), (i, j + 1)) for i in range(BOARD_SIZE) for j in range(BOARD_SIZE - 1)] + \
               [((i + 1, j), (i, j)) for i in range(BOARD_SIZE - 1)
                for j in range(BOARD_SIZE)]


class Best:
    def __init__(self, board: MyBoard):
        self.board = board

    # 在棋盘上进行一次操作
    def move(self, action):
        (x1, y1), (x2, y2) = action
        new_board = MyBoard(self.board.board.copy())
        new_board.change((x1, y1), (x2, y2))
        total_score, columns_eliminated = new_board.eliminate()
        while columns_eliminated.sum() != 0 and not (new_board.board[:BOARD_SIZE, :BOARD_SIZE] == 'nan').any():
            score, columns_eliminated = new_board.eliminate()
            total_score += score
        return new_board, total_score

    # 直接挑选最优解
    def select(self):
        value = 0
        choice = None
        for action in action_space:
            new_board, total_score = self.move(action)
            if total_score > value:
                value = total_score
                choice = action
        if choice is not None:
            return choice
        return action_space[np.random.randint(0, len(action_space))]


In [143]:
class Board_Getter:
    def __init__(self, size=BOARD_SIZE, board_num=N_ROWS//BOARD_SIZE, colors=np.array(COLORS)):
        '''
        Initialize board instance.
        Parameters:
        size(int): Size of each sub-board
        board_num(int): Number of sub-boards in the main board
        colors(np.array): Array of available colors
        '''
        self.size = size
        self.board_num = board_num
        self.colors = colors
        self.board = np.concatenate(
            [self.generate_board() for i in np.arange(board_num)])

    def generate_board(self):
        '''
        Generate a new sub-board.
        Returns:
        np.array: A new sub-board generated using the given rules.
        '''
        # Concatenate four color blocks, each of which has size*size/4 cells filled with the corresponding color.

        idx = np.arange(len(self.colors))
        np.random.shuffle(idx)
        remain = [1, ] * (self.size % len(self.colors)) + [0, ] * \
            (len(self.colors) - (self.size % len(self.colors)))
        a = np.hstack([np.full((self.size, self.size // len(self.colors) + remain[i]),
                               self.colors[idx[i]]) for i in range(len(self.colors))]).reshape(
            (self.size ** 2, 1))
        np.random.shuffle(a)  # Shuffle the order of color blocks.

        # Reshape sub-board array using the shuffled colors.
        new_array = a.reshape(self.size, self.size)

        # If there are three adjacent cells with the same color, randomly choose a different color for one of them
        while self.check(new_array):
            for i, j in self.check(new_array):
                new_array[i][j] = new_array[i - random.randint(
                    1, self.size - 1)][j - random.randint(1, self.size - 1)]

        # Return the generated sub-board.
        return new_array

    def check(self, array):
        '''
        Check if there are three adjacent cells in a sub-board that are filled with the same color.
        Parameters:
        array(np.array): A square sub-board array to be checked for repetition of colors.
        Returns:
        set: A set of tuples containing row and column indices of all cells that have three adjacent cells with
             the same color.
        '''
        repeats = set()

        # Traverse the rows
        for i in np.arange(1, self.size - 1):
            for j in np.arange(self.size):
                a = array[i - 1:i + 2, j]
                b = (a == array[i, j])
                if np.sum(b) == 3:
                    repeats.add((i, j))

        # Traverse the columns
        for i in np.arange(1, self.size - 1):
            for j in np.arange(self.size):
                a = array[j, i - 1:i + 2]
                b = (a == array[j, i])
                if np.sum(b) == 3:
                    repeats.add((j, i))

        # Return a set of tuple(s) which contain the location(s) of cells whose neighbors share the same color.
        return repeats

testboard = Board_Getter()


In [150]:
np.set_printoptions(threshold=np.inf)
np.random.seed(40)

In [151]:
def test():
    return Best(testboard).select()
test()

((2, 3), (2, 4))